In [2]:
##################################################################################################
### This script is ML Classification template, which should be applicable to most MLC projects ###
##################################################################################################

"""Structure of the script:
1.  Load all needed libraries and functions.
2.  Load data, do preliminary data exploration.
2.1 [Optional] Create more variables, delete variables.
3.  Deal with missing values, transform skewed variables.
4.  Trnasform features depending on their type. OHC.
5.  Create subsamples.
6.  Do scaling.
7.  Fit models, selecting hyperparameters via CV grid search.
8.  Evaluate performance of the selected models on test sample.
"""

'Structure of the script:\n1.  Load all needed libraries and functions.\n2.  Load data, do preliminary data exploration.\n2.1 [Optional] Create more variables, delete variables.\n3.  Deal with missing values, transform skewed variables.\n4.  Trnasform features depending on their type. OHC.\n5.  Create subsamples.\n6.  Do scaling.\n7.  Fit models, selecting hyperparameters via CV grid search.\n8.  Evaluate performance of the selected models on test sample.\n'

In [3]:
### 1.Load main libraries ###

import numpy as np
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn import svm
from sklearn.preprocessing import LabelBinarizer, LabelEncoder, OrdinalEncoder, MinMaxScaler, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, cross_val_predict, GridSearchCV, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from xgboost import XGBRegressor, XGBClassifier

pd.set_option('display.max_columns', 20)
pd.set_option('mode.chained_assignment', None)
pd.set_option('display.expand_frame_repr', False)

In [67]:
# Turn off warnings. Be warned!
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [4]:
def draw_histograms(df, variables, n_rows, n_cols):
    # stolen from https://stackoverflow.com/questions/29530355/plotting-multiple-histograms-in-grid
    fig=plt.figure()
    for i, var_name in enumerate(variables):
        ax=fig.add_subplot(n_rows,n_cols,i+1)
        df[var_name].hist(bins=10,ax=ax)
        ax.set_title(var_name+" Distribution")
    fig.tight_layout()  
    plt.show()

In [78]:
### 2.Load data ###

path = '../input/spaceship-titanic/train.csv'
df = pd.read_csv(path) 
print(df.shape)
#df.drop(columns=['Name', 'Ticket', 'Cabin'],inplace=True)
#df.loc[df.Age.isnull(),'Age'] = df.Age.median()
df.head()

(8693, 14)


,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [79]:
num_cols = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
cat_cols = ['HomePlanet','CryoSleep','VIP','Transported']

print(df[num_cols].describe())
print(df[cat_cols].apply(pd.Series.value_counts))
print(df.shape)

# sns.pairplot(df[['Survived', 'Pclass', 'Age', 'Fare']])
#draw_histograms(df, df.columns, 4, 3)

               Age   RoomService     FoodCourt  ShoppingMall           Spa        VRDeck
count  8514.000000   8512.000000   8510.000000   8485.000000   8510.000000   8505.000000
mean     28.827930    224.687617    458.077203    173.729169    311.138778    304.854791
std      14.489021    666.717663   1611.489240    604.696458   1136.705535   1145.717189
min       0.000000      0.000000      0.000000      0.000000      0.000000      0.000000
25%      19.000000      0.000000      0.000000      0.000000      0.000000      0.000000
50%      27.000000      0.000000      0.000000      0.000000      0.000000      0.000000
75%      38.000000     47.000000     76.000000     27.000000     59.000000     46.000000
max      79.000000  14327.000000  29813.000000  23492.000000  22408.000000  24133.000000
        HomePlanet  CryoSleep     VIP  Transported
Earth       4602.0        NaN     NaN          NaN
Europa      2131.0        NaN     NaN          NaN
Mars        1759.0        NaN     NaN         

In [80]:
#%% 2.5 Create more features ###

df[['Group_Id', 'Passenger_Id']] = df['PassengerId'].str.split('_', 1, expand=True)
df[['Deck', 'Room', 'Side']] = df['Cabin'].str.split('/', 2, expand=True)
print(df.dtypes)

df[['Group_Id', 'Passenger_Id', 'Room']] = df[['Group_Id', 'Passenger_Id', 'Room']].apply(pd.to_numeric)
df.drop(columns=['PassengerId', 'Cabin', 'Group_Id', 'Room', 'Name'], inplace=True)
df

PassengerId      object
HomePlanet       object
CryoSleep        object
Cabin            object
Destination      object
Age             float64
VIP              object
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Name             object
Transported        bool
Group_Id         object
Passenger_Id     object
Deck             object
Room             object
Side             object
dtype: object


,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Passenger_Id,Deck,Side
0,Europa,False,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False,1,B,P
1,Earth,False,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True,1,F,S
2,Europa,False,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False,1,A,S
3,Europa,False,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False,2,A,S
4,Earth,False,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,True,1,F,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,Europa,False,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,False,1,A,P
8689,Earth,True,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,False,1,G,S
8690,Earth,False,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,True,1,G,S
8691,Europa,False,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,False,1,E,S


In [81]:
#%% 3.Deal with missing values ###

df.info()
df.dropna(inplace=True)
df.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HomePlanet    8492 non-null   object 
 1   CryoSleep     8476 non-null   object 
 2   Destination   8511 non-null   object 
 3   Age           8514 non-null   float64
 4   VIP           8490 non-null   object 
 5   RoomService   8512 non-null   float64
 6   FoodCourt     8510 non-null   float64
 7   ShoppingMall  8485 non-null   float64
 8   Spa           8510 non-null   float64
 9   VRDeck        8505 non-null   float64
 10  Transported   8693 non-null   bool   
 11  Passenger_Id  8693 non-null   int64  
 12  Deck          8494 non-null   object 
 13  Side          8494 non-null   object 
dtypes: bool(1), float64(6), int64(1), object(6)
memory usage: 891.5+ KB


(6764, 14)

In [82]:
#%% Transform some skewed variables ###

df['RoomService'] = np.log1p(df.RoomService)
df['FoodCourt'] = np.log1p(df.FoodCourt)
df['ShoppingMall'] = np.log1p(df.ShoppingMall)
df['Spa'] = np.log1p(df.Spa)
df['VRDeck'] = np.log1p(df.VRDeck)

In [83]:
df['CryoSleep'] = df['CryoSleep'].astype(int)
df['VIP'] = df['VIP'].astype(int)
df['Transported'] = df['Transported'].astype(int)
df.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Passenger_Id,Deck,Side
0,Europa,0,TRAPPIST-1e,39.0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0,1,B,P
1,Earth,0,TRAPPIST-1e,24.0,0,4.700480,2.302585,3.258097,6.309918,3.806662,1,1,F,S
2,Europa,0,TRAPPIST-1e,58.0,1,3.784190,8.182280,0.000000,8.812248,3.912023,0,1,A,S
3,Europa,0,TRAPPIST-1e,33.0,0,0.000000,7.157735,5.918894,8.110728,5.267858,0,2,A,S
4,Earth,0,TRAPPIST-1e,16.0,0,5.717028,4.262680,5.023881,6.338594,1.098612,1,1,F,S


In [84]:
#%% 4.Transform features depending on their type ###

# this is very important for ML application, where there are hundreds of features.
# If there are less than 20 features, can use standard approach.
# my approach of tackling one feature a time is not scalable. 

# use intuition to trim range or ordinary variables 
# can skip this step in general, since it is not scalable when number of features grows.

# identify binary and categorical variables
df_uniques = pd.DataFrame([[i, len(df[i].unique())] for i in df.columns], columns=['Variable', 'Unique Values']).set_index('Variable')
print(df_uniques)

binary_variables = list(df_uniques[df_uniques['Unique Values'] == 2].index)
categorical_variables = list(df_uniques[(10 >= df_uniques['Unique Values']) & (df_uniques['Unique Values'] > 2)].index)
numeric_variables = list(set(df.columns) - set(categorical_variables) - set(binary_variables))
print('Binary variables are ', binary_variables)
print('Categorical variables are ', categorical_variables)
print('Numeric variables are ', numeric_variables)

              Unique Values
Variable                   
HomePlanet                3
CryoSleep                 2
Destination               3
Age                      80
VIP                       2
RoomService            1112
FoodCourt              1318
ShoppingMall           1000
Spa                    1162
VRDeck                 1125
Transported               2
Passenger_Id              8
Deck                      8
Side                      2
Binary variables are  ['CryoSleep', 'VIP', 'Transported', 'Side']
Categorical variables are  ['HomePlanet', 'Destination', 'Passenger_Id', 'Deck']
Numeric variables are  ['VRDeck', 'Spa', 'RoomService', 'Age', 'ShoppingMall', 'FoodCourt']


In [85]:
# ohc for binary variables #
lb = LabelBinarizer()
binary_variables.remove('Transported')
for column in binary_variables:
    df[column] = lb.fit_transform(df[column])

# ohc for categorical variables #
df = pd.get_dummies(df, columns = categorical_variables, drop_first=True)

print(df.shape)
print(df.head())
print(df.dtypes)

(6764, 28)
   CryoSleep   Age  VIP  RoomService  FoodCourt  ShoppingMall       Spa    VRDeck  Transported  Side  ...  Passenger_Id_6  Passenger_Id_7  Passenger_Id_8  Deck_B  Deck_C  Deck_D  Deck_E  Deck_F  Deck_G  Deck_T
0          0  39.0    0     0.000000   0.000000      0.000000  0.000000  0.000000            0     0  ...               0               0               0       1       0       0       0       0       0       0
1          0  24.0    0     4.700480   2.302585      3.258097  6.309918  3.806662            1     1  ...               0               0               0       0       0       0       0       1       0       0
2          0  58.0    1     3.784190   8.182280      0.000000  8.812248  3.912023            0     1  ...               0               0               0       0       0       0       0       0       0       0
3          0  33.0    0     0.000000   7.157735      5.918894  8.110728  5.267858            0     1  ...               0               0            

In [86]:
# %% 5.Creating subsamples ###

y = df['Transported']
X = df.drop(columns=['Transported'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=3)

In [62]:
# %% 6.scaling numeric variables ###

#draw_histograms(X_train, numeric_variables, 3, 3)

In [87]:
ss = StandardScaler()

for column in [numeric_variables]:
    X_train[column] = ss.fit_transform(X_train[column])
    X_test[column] = ss.transform(X_test[column])

In [88]:
X_train

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Side,HomePlanet_Europa,...,Passenger_Id_6,Passenger_Id_7,Passenger_Id_8,Deck_B,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G,Deck_T
2606,0,1.730936,0,-0.650297,0.559952,-0.637021,2.318293,-0.656248,0,1,...,0,0,0,1,0,0,0,0,0,0
2249,1,-0.474041,0,-0.650297,-0.665070,-0.637021,-0.683706,-0.656248,0,0,...,0,0,0,0,0,0,0,0,1,0
5432,0,-1.852152,0,-0.650297,-0.665070,-0.637021,-0.683706,-0.656248,0,0,...,0,0,0,0,0,0,0,0,1,0
5943,0,1.248597,1,-0.650297,1.930724,2.184811,1.201431,0.489831,0,1,...,0,0,0,0,1,0,0,0,0,0
2622,0,0.283920,0,-0.650297,-0.665070,1.296272,0.429809,1.763720,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1262,1,-0.818569,0,-0.650297,-0.665070,-0.637021,-0.683706,-0.656248,1,0,...,0,0,0,0,0,0,0,0,1,0
2168,1,-1.094191,0,-0.650297,-0.665070,-0.637021,-0.683706,-0.656248,1,0,...,0,0,0,0,0,0,0,0,1,0
4282,1,0.146109,0,-0.650297,-0.665070,-0.637021,-0.683706,-0.656248,1,1,...,0,0,0,1,0,0,0,0,0,0
2196,0,1.041881,1,-0.650297,1.729309,1.037917,1.893740,1.912421,1,1,...,0,0,0,0,0,1,0,0,0,0


In [89]:
####################
### 7.Fit models ###
####################

#%% Logistic regression ###

grid_values = {'penalty': ['l2'], 'C': list(np.arange(1,10,1))}
lr = LogisticRegression()
model_lr = GridSearchCV(lr, param_grid=grid_values, cv = 10)
model_lr.fit(X_train, y_train)
print(model_lr.best_score_, model_lr.best_params_)

# model_lr.predict(X_test)

0.7805248574021261 {'C': 6, 'penalty': 'l2'}


In [90]:
#%% KNN ###

grid_values = dict(n_neighbors=np.arange(10,41,2))
knnm = KNeighborsClassifier()   
model_knn = GridSearchCV(knnm, param_grid=grid_values, cv = 5)
model_knn.fit(X_train, y_train)
print(model_knn.best_score_, model_knn.best_params_)

0.7925149058291608 {'n_neighbors': 34}


In [91]:
#%% SVM ###

grid_values = {'C': np.arange(0.1, 1, 0.2)} 
svmm = svm.SVC(kernel='rbf')
model_svm = GridSearchCV(svmm, param_grid=grid_values, cv = 5)
model_svm.fit(X_train, y_train)
print(model_svm.best_score_, model_svm.best_params_)

0.8035211353121421 {'C': 0.7000000000000001}


In [92]:
#%% RF ###

# may look here: https://www.geeksforgeeks.org/hyperparameter-tuning/

grid_values = [{'max_depth': list(range(2, 9, 2)), 'max_features': list(np.arange(0.3,0.71,0.2))}]
rfc = RandomForestClassifier(random_state=42)
model_rf = GridSearchCV(rfc, grid_values, cv = 5, scoring='accuracy')
model_rf.fit(X_train, y_train)
print(model_rf.best_score_, model_rf.best_params_)

0.8059841894993343 {'max_depth': 8, 'max_features': 0.5}


In [ ]:
#%% XGBoost ###
# run this code only on Kaggle with GPU

estimator = XGBClassifier(
    nthread=4,
    seed=42,
    use_label_encoder=False
)

parameters = {
    'max_depth': range (2, 5, 1),
    'n_estimators': range(5, 50, 5),
    'learning_rate': [0.01, 0.05, 0.1, 0.15]
}

grid_search = GridSearchCV(
    estimator=estimator,
    param_grid=parameters,
    scoring = 'roc_auc',
    n_jobs = 10,
    cv = 10,
    verbose=True
)

grid_search.fit(X_train, y_train, eval_metric='rmse')
print(grid_search.best_score_, grid_search.best_params_)

In [93]:
#%% 8.Evaluate performance oos ###

yhat_lm = model_lr.predict(X_test)
yhat_knn = model_knn.predict(X_test)
yhat_svm = model_svm.predict(X_test)
yhat_rf = model_rf.predict(X_test)
#yhat_bt = grid_search.predict(X_test)
print('Accuracy of logistic regression is ', 1-(np.abs(yhat_lm-y_test)).mean())
print('Accuracy of KNN is ', 1-(np.abs(yhat_knn-y_test)).mean())
print('Accuracy of SVM is ', 1-(np.abs(yhat_svm-y_test)).mean())
print('Accuracy of RF is ', 1-(np.abs(yhat_rf-y_test)).mean())
#print('Accuracy of Boosted Tree is ', 1-(np.abs(yhat_bt-y_test)).mean())

Accuracy of logistic regression is  0.7872968980797637
Accuracy of KNN is  0.8050221565731167
Accuracy of SVM is  0.8124076809453471
Accuracy of RF is  0.7961595273264401
